# Project 2: PQC Lattice-Based Kyber (Real Library Demo)

This notebook demonstrates the **CRYSTALS-Kyber** KEM. We are using the "Noisy Lockbox" 📦 analogy.

* **Bob (The Receiver):** Creates a "magic lock" (public key 🔒) and a "magic key" (private key 🗝️).
* **Alice (The Sender):** Uses Bob's lock 🔒 to create a new secret 💎 and a "locked box" (`ciphertext`) 📦 containing it.
* **Bob:** Uses his key 🗝️ to open the box 📦 and get the secret 💎.

# Pre Requisites (run these commands first)

* `git clone --depth=1 https://github.com/open-quantum-safe/liboqs-python`
* `cd/liboqs-python`
* `pip install .`
* `export PYTHONPATH=$PYTHONPATH:/path/to/liboqs-python`


In [1]:
import oqs
import logging
from pprint import pformat
from sys import stdout

logger = logging.getLogger("KyberDemo")
logger.setLevel(logging.INFO)
logger.handlers.clear()
logger.addHandler(logging.StreamHandler(stdout))

logger.info("1. IMPORTS AND SETUP")
logger.info("liboqs version: %s", oqs.oqs_version())
logger.info("liboqs-python version: %s", oqs.oqs_python_version())
logger.info(
    "Enabled KEM mechanisms:\n%s",
    pformat(oqs.get_enabled_kem_mechanisms(), compact=True),
)

1. IMPORTS AND SETUP
liboqs version: 0.14.0
liboqs-python version: 0.14.0
Enabled KEM mechanisms:
('BIKE-L1', 'BIKE-L3', 'BIKE-L5', 'Classic-McEliece-348864',
 'Classic-McEliece-348864f', 'Classic-McEliece-460896',
 'Classic-McEliece-460896f', 'Classic-McEliece-6688128',
 'Classic-McEliece-6688128f', 'Classic-McEliece-6960119',
 'Classic-McEliece-6960119f', 'Classic-McEliece-8192128',
 'Classic-McEliece-8192128f', 'Kyber512', 'Kyber768', 'Kyber1024', 'ML-KEM-512',
 'ML-KEM-768', 'ML-KEM-1024', 'sntrup761', 'FrodoKEM-640-AES',
 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES', 'FrodoKEM-976-SHAKE',
 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE')


## Step 2: Bob's Side (The Receiver)

We begin by setting up Bob. He is the **receiver**. He needs to generate his key pair.

* **Public Key 🔒:** The "magic lock" he will give to Alice.
* **Private Key 🗝️:** The "magic key" he keeps secret.

In [2]:
logger.info("\n2. BOB'S SIDE (THE RECEIVER)")

# We select the KEM algorithm
kemalg = "Kyber512"

# Bob creates his KEM object
kem_bob = oqs.KeyEncapsulation(kemalg)

# Bob generates his key pair
logger.info("Bob (Receiver) is generating his key pair...")
public_key_bob = kem_bob.generate_keypair()

logger.info("Bob has his public key 🔒 and private key 🗝️.")
logger.info("Bob's Public Key (first 30 bytes):\n%s...", public_key_bob.hex()[:30])


2. BOB'S SIDE (THE RECEIVER)
Bob (Receiver) is generating his key pair...
Bob has his public key 🔒 and private key 🗝️.
Bob's Public Key (first 30 bytes):
cd4c642d751669648e88d63df9d9ac...


## Step 3: Alice's Side (The Sender)

Now, Alice (the **sender**) gets Bob's public key 🔒.

Her goal is to "encapsulate" a new secret. The `.encap_secret()` function is magic:
1.  It creates a new, random secret 💎 (`shared_secret_alice`).
2.  It uses Bob's lock 🔒 to create a "locked box" 📦 (`ciphertext`).
3.  It gives *both* back to Alice.

In [3]:
logger.info("\n3. ALICE'S SIDE (THE SENDER)")

# Alice creates her own KEM object
kem_alice = oqs.KeyEncapsulation(kemalg)
logger.info("Alice has Bob's public key 🔒.")

# Alice runs the "Encapsulation" step
logger.info("Alice is creating a new secret and locking it...")
ciphertext, shared_secret_alice = kem_alice.encap_secret(public_key_bob)

logger.info("Alice has her new secret (for herself).")
logger.info("Alice's Secret (first 30 bytes): \n%s...", shared_secret_alice.hex()[:30])
logger.info("\nAlice sends the 'locked box' 📦 to Bob.")
logger.info("Ciphertext (first 30 bytes): \n%s...", ciphertext.hex()[:30])


3. ALICE'S SIDE (THE SENDER)
Alice has Bob's public key 🔒.
Alice is creating a new secret and locking it...
Alice has her new secret (for herself).
Alice's Secret (first 30 bytes): 
3e824fa5f286cc6496b6194c9602f6...

Alice sends the 'locked box' 📦 to Bob.
Ciphertext (first 30 bytes): 
470f4cfde06dc1a6182fd162af86ed...


## Step 4: Bob's Side (The "Decapsulation")

Bob receives the `ciphertext` 📦 from Alice. He now uses his **private key 🗝️** (which his `kem_bob` object has been holding) to "decapsulate" (unlock) the box.

In [4]:
logger.info("\n4. BOB'S SIDE (THE DECAPSULATION)")
logger.info("Bob receives the 'locked box' 📦.")
logger.info("Bob uses his private key 🗝️ to open it...")

# Bob uses his KEM object (which has his private key)
# to decrypt the ciphertext.
shared_secret_bob = kem_bob.decap_secret(ciphertext)

logger.info("Bob has retrieved the secret from the box.")
logger.info("Bob's Retrieved Secret (first 30 bytes): \n%s...", shared_secret_bob.hex()[:30])


4. BOB'S SIDE (THE DECAPSULATION)
Bob receives the 'locked box' 📦.
Bob uses his private key 🗝️ to open it...
Bob has retrieved the secret from the box.
Bob's Retrieved Secret (first 30 bytes): 
3e824fa5f286cc6496b6194c9602f6...


## Step 5: The Test Case (The Proof)

This is the "Aha!" moment. We check if the secret Alice *created* is identical to the secret Bob *retrieved*.

In [5]:
logger.info("\n5. THE TEST CASE")

# We compare the two secrets
secrets_match = (shared_secret_alice == shared_secret_bob)

logger.info("Does Alice's secret == Bob's secret?")
logger.info("Shared secrets coincide: %s", secrets_match)

if secrets_match:
    logger.info("   >>> SUCCESS! <<<")
    logger.info("   Alice has successfully sent a quantum-resistant secret to Bob.")
else:
    logger.info("   >>> FAILURE! <<<")
    
print("\n\nEnd of PQC-Kyber Simulation")


5. THE TEST CASE
Does Alice's secret == Bob's secret?
Shared secrets coincide: True
   >>> SUCCESS! <<<
   Alice has successfully sent a quantum-resistant secret to Bob.


End of PQC-Kyber Simulation


In [1]:
import time
import oqs
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes


ITERATIONS = 10  
RSA_KEY_SIZE = 2048
KYBER_ALG = "Kyber512"

print(f"")
print(f"   Speed test: RSA-{RSA_KEY_SIZE} vs. {KYBER_ALG}")
print(f"=================================================================")
print(f"Running {ITERATIONS} rounds. Streaming results live...\n")


rsa_keygen_times = []
rsa_enc_times = []
rsa_dec_times = []
kyber_keygen_times = []
kyber_enc_times = []
kyber_dec_times = []

print(f"--- ROUND 1: RSA ({RSA_KEY_SIZE}-bit) ---")
print(f"{'Run':<5} | {'KeyGen (s)':<12} | {'Encrypt (s)':<12} | {'Decrypt (s)':<12} | {'Status'}")
print("-" * 65)

for i in range(ITERATIONS):
    # A. RSA Key Generation
    start = time.time()
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=RSA_KEY_SIZE)
    public_key = private_key.public_key()
    t_keygen = time.time() - start
    rsa_keygen_times.append(t_keygen)

    # B. RSA Encryption
    message = b"Secret Message"
    start = time.time()
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    t_enc = time.time() - start
    rsa_enc_times.append(t_enc)

    # C. RSA Decryption
    start = time.time()
    decrypted = private_key.decrypt(
        ciphertext,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    t_dec = time.time() - start
    rsa_dec_times.append(t_dec)
    
    # Verify
    status = "✅" if decrypted == message else "❌"
    
    # Live Print
    print(f"{i+1:<5} | {t_keygen:.5f}      | {t_enc:.5f}      | {t_dec:.5f}      | {status}")

print("-" * 65)
print("RSA Testing Complete.\n")
time.sleep(1)



print(f"--- ROUND 2: KYBER ({KYBER_ALG}) ---")
print(f"{'Run':<5} | {'KeyGen (s)':<12} | {'Encap (s)':<12} | {'Decap (s)':<12} | {'Status'}")
print("-" * 65)

# Initialize Kyber (We create the object once, but regenerate keys every loop)
kem = oqs.KeyEncapsulation(KYBER_ALG)

for i in range(ITERATIONS):
    # A. Kyber Key Generation
    start = time.time()
    public_key_kyber = kem.generate_keypair()
    t_keygen = time.time() - start
    kyber_keygen_times.append(t_keygen)

    # B. Kyber Encapsulation
    start = time.time()
    ciphertext_kyber, shared_secret_alice = kem.encap_secret(public_key_kyber)
    t_enc = time.time() - start
    kyber_enc_times.append(t_enc)

    # C. Kyber Decapsulation
    start = time.time()
    shared_secret_bob = kem.decap_secret(ciphertext_kyber)
    t_dec = time.time() - start
    kyber_dec_times.append(t_dec)
    
    # Verify
    status = "✅" if shared_secret_alice == shared_secret_bob else "❌"

    # Live Print
    print(f"{i+1:<5} | {t_keygen:.5f}      | {t_enc:.5f}      | {t_dec:.5f}      | {status}")

print("-" * 65)
print("Kyber Testing Complete.\n")

avg_rsa_k = sum(rsa_keygen_times) / ITERATIONS
avg_ky_k = sum(kyber_keygen_times) / ITERATIONS

avg_rsa_e = sum(rsa_enc_times) / ITERATIONS
avg_ky_e = sum(kyber_enc_times) / ITERATIONS

avg_rsa_d = sum(rsa_dec_times) / ITERATIONS
avg_ky_d = sum(kyber_dec_times) / ITERATIONS

print("\n" + "="*60)
print("   FINAL SCOREBOARD (AVERAGE TIMES)")
print("="*60)
print(f"{'OPERATION':<20} | {'RSA (sec)':<12} | {'KYBER (sec)':<12} | {'SPEEDUP'}")
print("-" * 60)

print(f"{'Key Generation':<20} | {avg_rsa_k:.5f}      | {avg_ky_k:.5f}        | {avg_rsa_k/avg_ky_k:.1f}x FASTER")
print(f"{'Encap/Encrypt':<20} | {avg_rsa_e:.5f}      | {avg_ky_e:.5f}        | {avg_rsa_e/avg_ky_e:.1f}x FASTER")
print(f"{'Decap/Decrypt':<20} | {avg_rsa_d:.5f}      | {avg_ky_d:.5f}        | {avg_rsa_d/avg_ky_d:.1f}x FASTER")
print("-" * 60)
print("Conclusion: Kyber is computationally lighter and significantly faster.")


   Speed test: RSA-2048 vs. Kyber512
Running 10 rounds. Streaming results live...

--- ROUND 1: RSA (2048-bit) ---
Run   | KeyGen (s)   | Encrypt (s)  | Decrypt (s)  | Status
-----------------------------------------------------------------
1     | 0.12174      | 0.00214      | 0.00252      | ✅
2     | 0.02884      | 0.00012      | 0.00222      | ✅
3     | 0.05418      | 0.00010      | 0.00225      | ✅
4     | 0.08332      | 0.00011      | 0.00220      | ✅
5     | 0.15422      | 0.00011      | 0.00219      | ✅
6     | 0.06838      | 0.00011      | 0.00217      | ✅
7     | 0.02150      | 0.00008      | 0.00219      | ✅
8     | 0.02344      | 0.00009      | 0.00217      | ✅
9     | 0.03709      | 0.00012      | 0.00233      | ✅
10    | 0.04251      | 0.00010      | 0.00227      | ✅
-----------------------------------------------------------------
RSA Testing Complete.

--- ROUND 2: KYBER (Kyber512) ---
Run   | KeyGen (s)   | Encap (s)    | Decap (s)    | Status
-------------------------